<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3:
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package in Python

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('DROP TABLE houses;')

# to avoid the error add if exists:
# c.execute('DROP TABLE houses if exists ;')


# Save (commit) the changes:
conn.commit()

In [3]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> `c.execute(sql_command, values)`

In [4]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> `c.executemany(sql_command, values)`

In [5]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
#
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Select all rows from houses

In [6]:
# A
query = 'Select * from houses'
c.execute(query)
c.fetchall()
#pd.DataFrame(c.fetchall())

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

ℹ️ My note:
- `genfromtxt`: Load data from a text file, with missing values handled as specified.
- `numpy.loadtxt`: Equivalent function when no data is missing.
 

In [3]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('../DATA/housing_data.csv', dtype = 'i8',
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [8]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None) # at position 0 insert None --> This is a Python list not a numpy array

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [9]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [10]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [11]:
c.execute("select count(*) from houses")
c.fetchall()

[(51,)]

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

*??? d1 below is still a list*

In [12]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()

d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [13]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

ANSWER: The clause above deletes items with id (column 0, unnamed) is 52 or 53

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [14]:
# ANSWER
query = "SELECT * FROM houses WHERE bdrms=4"
c.execute(query)
c.fetchall()  # What does this line do

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [15]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behavior or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [5]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Reading/writing data from/into a database

Data in Pandas can be loaded into a relational database.

#### Read database into dataframe
If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [17]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


#### Write DataFrame to database

We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [18]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

47

Run a query to get the average price of each house size from this table:

In [19]:
#ANSWER

q = "select sqft, avg(price) from houses_pandas group by sqft"
c.execute(q)
c.fetchall()

[(852, 179900.0),
 (1000, 169900.0),
 (1100, 249900.0),
 (1200, 299000.0),
 (1203, 239500.0),
 (1236, 199900.0),
 (1239, 229900.0),
 (1268, 259900.0),
 (1320, 299900.0),
 (1380, 212000.0),
 (1416, 232000.0),
 (1427, 198999.0),
 (1437, 249900.0),
 (1458, 464500.0),
 (1494, 242500.0),
 (1534, 314900.0),
 (1600, 329900.0),
 (1604, 242900.0),
 (1664, 368500.0),
 (1767, 252900.0),
 (1811, 285900.0),
 (1839, 349900.0),
 (1852, 299900.0),
 (1888, 255000.0),
 (1890, 329999.0),
 (1940, 239999.0),
 (1962, 259900.0),
 (1985, 299900.0),
 (2000, 347000.0),
 (2040, 314900.0),
 (2104, 399900.0),
 (2132, 345000.0),
 (2162, 287000.0),
 (2200, 475000.0),
 (2238, 329900.0),
 (2300, 449900.0),
 (2400, 369000.0),
 (2526, 469000.0),
 (2567, 314000.0),
 (2609, 499998.0),
 (2637, 299900.0),
 (3000, 539900.0),
 (3031, 599000.0),
 (3137, 579900.0),
 (3890, 573900.0),
 (4215, 549000.0),
 (4478, 699900.0)]

In [20]:
# This would be better on binned data
data.describe()

,sqft,bdrms,age,price
count,47.000000,47.000000,47.000000,47.000000
mean,2000.680851,3.170213,42.744681,340412.659574
std,794.702354,0.760982,22.873440,125039.899586
min,852.000000,1.000000,5.000000,169900.000000
25%,1432.000000,3.000000,24.500000,249900.000000
50%,1888.000000,3.000000,44.000000,299900.000000
75%,2269.000000,4.000000,61.500000,384450.000000
max,4478.000000,5.000000,79.000000,699900.000000


In [21]:
data['size'] = pd.cut(data.sqft, bins = [851, 1432, 1888, 2269, 4478]).astype('str')
data.head()

,sqft,bdrms,age,price,size
0,2104,3,70,399900,"(1888, 2269]"
1,1600,3,28,329900,"(1432, 1888]"
2,2400,3,44,369000,"(2269, 4478]"
3,1416,2,49,232000,"(851, 1432]"
4,3000,4,75,539900,"(2269, 4478]"


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sqft    47 non-null     int64 
 1   bdrms   47 non-null     int64 
 2   age     47 non-null     int64 
 3   price   47 non-null     int64 
 4   size    47 non-null     object
dtypes: int64(4), object(1)
memory usage: 2.0+ KB


In [23]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False) # rewrite database with new column
q = "select size, avg(price) from houses_pandas group by size"
c.execute(q)
c.fetchall()


[('(1432, 1888]', 304725.0),
 ('(1888, 2269]', 329863.45454545453),
 ('(2269, 4478]', 495283.1666666667),
 ('(851, 1432]', 230899.91666666666)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

To modify the data before loading it.  Doing changes to data is easier with Pandas than with SQL.

2. When would you want to use Pandas on a dataset retrieved from a database?

When you want to do complex analyses on the data.  That is also easier in Pandas than in SQL.

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [6]:
# Reading CSV to Dataframe
orders = pd.read_csv('../DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [25]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [26]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Convert column names to lowercase and Replace Space with Underscore in Column Names

In [27]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [28]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes

In [29]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [30]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [31]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')

# Save these two dataframes as a table in sqlite

# data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)
orders.to_sql('orders', con = db_connection, if_exists='replace', index=False)
orders_break_down.to_sql('orders_break_down', con=db_connection, if_exists='replace', index=False)

8047

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [32]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT *
        FROM orders_break_down
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
5,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies
6,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies
7,ES-2011-5433855,"Bush Floating Shelf Set, Pine",0.1,155.0,1,Furniture
8,ES-2011-5433855,"Accos Thumb Tacks, Assorted Sizes",0.0,33.0,3,Office Supplies
9,ES-2011-5433855,"Smead Lockers, Industrial",0.1,716.0,4,Office Supplies


In [33]:
# Select Number of Distinct Orders for Each Customer
query = """
        SELECT DISTINCT customer_name, order_id, order_date
        FROM orders
        LIMIT 15
"""

distinct_orders = sql.read_sql(query, con=db_connection)
distinct_orders

,customer_name,order_id,order_date
0,Eugene Moren,IT-2011-3647632,2011-01-01
1,Dorothy Dickinson,ES-2011-4869686,2011-01-03
2,Arthur Prichep,ES-2011-4939443,2011-01-04
3,Grant Thornton,IT-2011-2942451,2011-01-04
4,Michael Granlund,ES-2011-3848439,2011-01-05
5,Dave Poirier,ES-2011-5433855,2011-01-07
6,Darren Powers,IT-2011-4546695,2011-01-08
7,Eric Murdock,ES-2011-1138719,2011-01-11
8,Mick Brown,ES-2011-1466305,2011-01-11
9,Dorothy Dickinson,ES-2011-4359424,2011-01-11


#### 6. Select Number of Customers for Each Country

In [34]:
# Select Number of Customers for Each Country
query = """
        select country, count(customer_name) as num_customers
        from orders
        group by country
        order by num_customers desc
"""

customers_per_country = sql.read_sql(query, con=db_connection)
customers_per_country

,country,num_customers
0,France,991
1,Germany,806
2,United Kingdom,700
3,Italy,493
4,Spain,403
5,Netherlands,194
6,Austria,135
7,Sweden,100
8,Belgium,68
9,Ireland,50


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [35]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
query = """
        select order_id, discount, sales, quantity
        from orders_break_down
        order by order_id
"""
orders_summary = sql.read_sql(query, con=db_connection)
orders_summary.head()

,order_id,discount,sales,quantity
0,ES-2011-1001989,0.1,229.0,9
1,ES-2011-1010958,0.0,58.0,2
2,ES-2011-1010958,0.0,140.0,6
3,ES-2011-1012469,0.0,148.0,3
4,ES-2011-1043483,0.0,224.0,11


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [36]:
# Select discount, sales, quantity, total price for Each Order Id from orders_break_down Table

query = """
        select order_id, discount, sales, quantity, sales * quantity - discount as total_price
        from orders_break_down
        order by order_id
"""
orders_summary = sql.read_sql(query, con=db_connection)
orders_summary.head()

,order_id,discount,sales,quantity,total_price
0,ES-2011-1001989,0.1,229.0,9,2060.9
1,ES-2011-1010958,0.0,58.0,2,116.0
2,ES-2011-1010958,0.0,140.0,6,840.0
3,ES-2011-1012469,0.0,148.0,3,444.0
4,ES-2011-1043483,0.0,224.0,11,2464.0


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [37]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

query = """
        select *, sales * quantity - discount as total_price from orders_break_down
        where total_price > 100
"""

exceeds_100 = sql.read_sql(query, con=db_connection)
exceeds_100.head()

,order_id,product_name,discount,sales,quantity,category,total_price
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies,134.5
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture,5978.0
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies,420.0
3,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies,270.0
4,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies,828.0


#### 8. Select All Customers And The Products They Have Bought

In [38]:
print(orders.columns)
print()
print(orders_break_down.columns)

Index(['order_id', 'order_date', 'customer_name', 'country'], dtype='object')

Index(['order_id', 'product_name', 'discount', 'sales', 'quantity',
       'category'],
      dtype='object')


In [39]:
# Select All Order Ids, Customers And The Products (product_name) They Have Bought

query = """
        select orders.order_id, customer_name, product_name
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        order by customer_name, orders.order_id
"""

# NOTE! Had to specify which table to use order_id from in the display to avoid "ambiguous column name" error

customers_products = sql.read_sql(query, con=db_connection)
customers_products.head(20)

,order_id,customer_name,product_name
0,ES-2011-3528133,Aaron Bergman,"Apple Office Telephone, Cordless"
1,ES-2011-3528133,Aaron Bergman,"Elite Box Cutter, High Speed"
2,ES-2011-3528133,Aaron Bergman,"Harbour Creations Round Labels, Laser Printer ..."
3,ES-2011-4146320,Aaron Bergman,"Binney & Smith Markers, Fluorescent"
4,ES-2011-4146320,Aaron Bergman,"Novimex Shipping Labels, Alphabetical"
5,ES-2011-4184901,Aaron Bergman,"Elite Letter Opener, High Speed"
6,ES-2011-4184901,Aaron Bergman,"Tenex Light Bulb, Duo Pack"
7,ES-2012-2621001,Aaron Bergman,"Konica Receipt Printer, Durable"
8,ES-2012-4291867,Aaron Bergman,"Office Star Bag Chairs, Black"
9,ES-2012-4291867,Aaron Bergman,"Okidata Calculator, Durable"


#### 9.A Select Number of 'Furniture' Orders For Each Country

In [40]:
unique_product_types = list(orders_break_down['category'].unique())
unique_product_types

['Office Supplies', 'Furniture', 'Technology']

In [41]:
# Select Number of 'Furniture' Orders For Each Country
query = """
        select country, count(orders.order_id) as num_furniture_orders
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        where category = "Furniture"
        group by country
        order by num_furniture_orders desc
"""

furniture_orders_by_country = sql.read_sql(query, con=db_connection)
furniture_orders_by_country

,country,num_furniture_orders
0,France,299
1,Germany,264
2,United Kingdom,180
3,Italy,136
4,Spain,123
5,Netherlands,76
6,Austria,40
7,Sweden,36
8,Belgium,20
9,Finland,16


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [42]:
# Select Number of 'Furniture' Orders For The Country Denmark

query = """
        select country, count(orders.order_id) as num_furniture_orders
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        where category = "Furniture" and country="Denmark"
        
"""

dk_furniture_orders = sql.read_sql(query, con=db_connection)
dk_furniture_orders

,country,num_furniture_orders
0,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [43]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country

# Change to use a CASE statement
query = """
        select country, sum(sales), discount > 0 as has_discount
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        group by country, has_discount
        order by country, has_discount
        """

sales_by_discount = sql.read_sql(query, con=db_connection)
sales_by_discount

,country,sum(sales),has_discount
0,Austria,79382.0,0
1,Belgium,42320.0,0
2,Denmark,7763.0,1
3,Finland,20702.0,0
4,France,129791.0,0
5,France,479892.0,1
6,Germany,245424.0,0
7,Germany,243257.0,1
8,Ireland,15998.0,1
9,Italy,174097.0,0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [44]:
# Select Total Quantity, Total Sales for Each Country
query = """
        select country, sum(quantity) as total_quantity, sum(sales) as total_sales
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        group by country
        """

sales_qty_by_country = sql.read_sql(query, con=db_connection)
sales_qty_by_country

,country,total_quantity,total_sales
0,Austria,973,79382.0
1,Belgium,532,42320.0
2,Denmark,204,7763.0
3,Finland,201,20702.0
4,France,7329,609683.0
5,Germany,6179,488681.0
6,Ireland,392,15998.0
7,Italy,3612,252742.0
8,Netherlands,1526,70313.0
9,Norway,261,20529.0


#### 11.B Select Top 3 Countries and Quantities Based on Total Sales

In [45]:
# Select Top 3 Countries and Quantities Based on Total Sales

query = """
        select country, sum(quantity) as total_quantity, sum(sales) as total_sales
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        group by country
        order by total_sales desc
        limit 3
        """

sales_qty_by_country_top_3 = sql.read_sql(query, con=db_connection)
sales_qty_by_country_top_3

,country,total_quantity,total_sales
0,France,7329,609683.0
1,Germany,6179,488681.0
2,United Kingdom,4917,420497.0


#### 11.C Select Bottom 3 Countries and Sales Based On Total Quantities

In [46]:
# Select Bottom 3 Countries and Sales Based On Total Quantities
# same as above but sort total sales ascending
query = """
        select country, sum(quantity) as total_quantity, sum(sales) as total_sales
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        group by country
        order by total_sales
        limit 3
        """

sales_qty_by_country_bottom_3 = sql.read_sql(query, con=db_connection)
sales_qty_by_country_bottom_3

,country,total_quantity,total_sales
0,Denmark,204,7763.0
1,Portugal,286,15106.0
2,Ireland,392,15998.0


#### 12. Select Average Sales By Category For The Country 'France'

In [47]:
# Select Average Sales By Category For The Country 'France'
query = """
        select country, avg(sales) as avg_sales
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        where country="France"
        """

avg_sales_france = sql.read_sql(query, con=db_connection)
avg_sales_france

,country,avg_sales
0,France,318.206159


#### 13. Select Country, Category and Average Sales Where Average Sales is The Highest

In [48]:
# Select Country, Category and Average Sales Where Average Sales is The Highest


# nested query
# where avg_sales = (select max(avg sales) from ...)
query = """
        select country, category, avg(sales) as avg_sales
        from orders join orders_break_down on orders.order_id = orders_break_down.order_id
        group by country, category
        order by avg_sales desc
        limit 1
        """

top_avg_sales = sql.read_sql(query, con=db_connection)
top_avg_sales

,country,category,avg_sales
0,Switzerland,Technology,902.846154


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



